In [1]:
#############################################################################################################
# Extracting chemical-disease associations from the biological literature
# R214: Main Practical
# Jan Ondras (jo356), Trinity College
#############################################################################################################
###################################################################################################
# Evaluate co-occurrences, loading data from saved calculated cooccurrences
'''
# Sort by each co-occurrence measure & Extract top 10 ranked
# Choose most appropriate concept names for best ranked pairs (i.e. print all possible MeSH entry names for concepts from top 10)
# Evaluate ranking similarity between the 4 measures
# 	(A) Top 10 ranking list overlap between various measures
# 	(B) Normalised Spearman's Footrule
# Find the top 10 ranked chem-dise pairs in the dataset of chemically-induced diseases (CID) (BioCreative V CDR)
# Examine chemical-disease relation type: print table & calculate what kind of relations are best found by which measure
# Extract unique chemical-disease pairs (CDPs) from all top 10 rankings
'''
###################################################################################################

import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
import time
import glob

# Load data saved above
cnt_data = np.load('./../Dataset/PubMed_counts.npz')

# concept IDs and counts
cooc =     cnt_data['cooc']     
chemCnts = cnt_data['chemCnts']
diseCnts = cnt_data['diseCnts']
# N_sentences = cnt_data['N_sentences']
N_sentences = 10573978

print "Total # unique chemical-disease pairs (CDPs):", len(cooc)
print 

# Restore original dictionaries: chem/dise -> count
chemCnts_dict = dict(chemCnts) 
diseCnts_dict = dict(diseCnts)

print "Examples of loaded counts data:"
print chemCnts[-5:]
print cooc[-5:]

###################################################################################################
# Load MESH concepts: <name, type (Chemical or Disease), code>
MESH = np.loadtxt('./../Dataset/CDR_MeSH.tsv', delimiter='\t', skiprows=0, dtype=str) # <name, label, code>
# Construct mapping dictionary: MESHcode -> MESH full concept name
# for conversion of MESH code to human readable full concept name (see next cell)
better_concept_names = [      # To show more easily interpretable concept name. Determined by investigation in a later cell (below)
    ['D003920', 'diabetes'], 
    ['D003866', 'depression'], 
    ['D009369', 'cancer'], 
    ['D000431', 'alcohol'], 
    ['D002251', 'carbon tetrachloride'], 
    ['D011020', 'pneumocystis pneumonia'],
    ['D010300', "Parkinson's disease"], 
    ['D010622', 'phencyclidine'], 
    ['D004409', 'abnormal movements'],         # orofacial dyskinesia / palpebral twitching
    ['D056486', 'toxic hepatitis'], 
    ['D006509', 'hepatitis B'],
    ['D001943', 'breast cancer'],
    ['D019259', 'lamivudine'],
    ['D000082', 'paracetamol'], 
    ['D007980', 'levodopa'], 
    ['D008774', 'methylphenidate'],
    ['D004317', 'doxorubicin'], 
    ['D003622', 'dapsone'], 
    ['D000244', 'adenosine diphosphate'],  # added manually
    ['D007538', 'isoniazid']
]
MESHcodesToNames = {}
MESHcodesToAllNames = {}
for r in MESH:
    MESHcodesToNames[r[2]] = r[0]
    if r[2] not in MESHcodesToAllNames:
        MESHcodesToAllNames[r[2]] = [r[0]]
    else:
        MESHcodesToAllNames[r[2]].append( r[0] )
for ID, n in better_concept_names:
    MESHcodesToNames[ID] = n
###################################################################################################
# Calculate all co-occurrence measures, store as dictionary: chemID+diseID -> (chemCnt, diseCnt, list of measures)

cooc_measures_names = ['SCC', 'NPMI', 'SCP', 'DC'] # presented in this order
cooc_measures_dict = {}

for chemDisePairIDs, cooc_CNT in cooc:
    [chemID, diseID] = chemDisePairIDs.split('+')
    N_cd = float(cooc_CNT)
    N_c = float(chemCnts_dict[chemID])
    N_d = float(diseCnts_dict[diseID])
    
    # 1.) Simple cooc count (SCC)
    SCC = N_cd
    # 2.) Normalised Pointwise Mutual Information (NPMI)
    NPMI = np.log( N_sentences * N_cd / (N_c * N_d) ) / np.log( N_sentences / N_cd )
    # 3.) Symmetric Conditional Probability (SCP)
    SCP = N_cd*N_cd / (N_c * N_d)
    # 4.) Dice coefficient (DC)
    DC = 2. * N_cd / (N_c + N_d) 
    
    cooc_measures_dict[chemDisePairIDs] = (N_c, N_d, [SCC, NPMI, SCP, DC])

print "Sample from cooc_measures_dict:\n\t", cooc_measures_dict[cooc_measures_dict.keys()[0]]

###################################################################################################
# Sort according to each measure: srt_cooc['NPMI'] gives data sorted by NPMI
# Take best N scores

BEST_N = 10 
srt_cooc = {} # best 10
srt_cooc_all = {} # all
for cmID, cmn in enumerate(cooc_measures_names):
    srt_cooc_all[cmn] = sorted(cooc_measures_dict.items(), key=lambda x: x[1][2][cmID], reverse=True)
    srt_cooc[cmn] = srt_cooc_all[cmn][:BEST_N] # record top 10
#     srt_cooc[cmn] = np.array(sorted(cooc_measures_dict.items(), key=lambda x: x[1][2][cmID] ))#[:BEST_N]
#     print srt_cooc[cmn][0]
print "Sorted by all measures."
print srt_cooc['SCC'][0]

Total # unique chemical-disease pairs (CDPs): 46137

Examples of loaded counts data:
[['D017665' '4058']
 ['C032171' '103']
 ['C010013' '6']
 ['D000547' '1916']
 ['D019257' '1558']]
[['D048288+D059350' '1']
 ['D015740+D020258' '4']
 ['C534628+D015878' '1']
 ['D000617+D014947' '4']
 ['D008774+D005076' '2']]
Sample from cooc_measures_dict:
	(5051.0, 620.0, [1.0, 0.075235109373321035, 3.193235450022672e-07, 0.00035267148651031563])
Sorted by all measures.
('D005947+D003920', (90297.0, 16278.0, [5062.0, 0.47027878248193927, 0.017432911114282308, 0.09499413558526859]))


In [2]:
###################################################################################################
# Print table of top 10 ranked for each measure
###################################################################################################

for cmID, cmn in enumerate(cooc_measures_names):

    cooc_concepts_names = []
    list_N_c = []
    list_N_d = []
    list_current_measure = []
    list_all_measures = []
    
    for chemDisePairIDs, (N_c, N_d, measures) in srt_cooc[cmn]:

        [chemID, diseID] = chemDisePairIDs.split('+')
        cooc_concepts_names.append( MESHcodesToNames[chemID] + ' + ' + MESHcodesToNames[diseID] )
        
        list_N_c.append( N_c )
        list_N_d.append( N_d )
        list_current_measure.append( measures[cmID] )
        list_all_measures.append( measures )
    list_N_c = np.array(list_N_c)
    list_N_d = np.array(list_N_d)
    list_current_measure = np.array(list_current_measure)
    list_all_measures = np.array(list_all_measures)
    
    # Table options
    # 1.)
    tab_data = np.vstack((list_N_c, list_N_d)).T
    tab_data = np.concatenate( (tab_data, list_all_measures), axis=1)
    tab_header = ['Chemical ($c$) + Disease ($d$)'] + ['$N_c$', '$N_d$'] + cooc_measures_names
    fmt = ("", ".0f", ".0f", ".0f", ".4f", ".4f", ".4f")
    print "Ordered by measure: ", cmn
    print tabulate(tab_data, 
                   headers=tab_header, tablefmt='fancy_grid', showindex=cooc_concepts_names,
                   numalign='center', floatfmt=fmt)
    print tabulate(tab_data, 
                   headers=tab_header, tablefmt='latex_booktabs', showindex=cooc_concepts_names,
                   numalign='center', floatfmt=fmt)

    # 2.)
#     tab_data = np.vstack((list_N_c, list_N_d, list_current_measure)).T
#     tab_header = ['Chemical ($c$) + Disease ($d$)'] + ['$N_c$', '$N_d$'] + [cmn]
#     if cmID == 0:
#         fmt = ("", ".0f", ".0f", ".0f")
#     else:
#         fmt = ("", ".0f", ".0f", ".4f")
#     print "Ordered by measure: ", cmn
#     print tabulate(tab_data, 
#                    headers=tab_header, tablefmt='fancy_grid', showindex=cooc_concepts_names,
#                    numalign='center', floatfmt=fmt)

Ordered by measure:  SCC
╒════════════════════════════════════════╤═════════╤═════════╤═══════╤════════╤════════╤════════╕
│ Chemical ($c$) + Disease ($d$)         │  $N_c$  │  $N_d$  │  SCC  │  NPMI  │  SCP   │   DC   │
╞════════════════════════════════════════╪═════════╪═════════╪═══════╪════════╪════════╪════════╡
│ glucose + diabetes                     │  90297  │  16278  │ 5062  │ 0.4703 │ 0.0174 │ 0.0950 │
├────────────────────────────────────────┼─────────┼─────────┼───────┼────────┼────────┼────────┤
│ cisplatinum + cancer                   │  36345  │  48709  │ 4175  │ 0.4104 │ 0.0098 │ 0.0982 │
├────────────────────────────────────────┼─────────┼─────────┼───────┼────────┼────────┼────────┤
│ doxorubicin + cancer                   │  38219  │  48709  │ 4075  │ 0.3997 │ 0.0089 │ 0.0938 │
├────────────────────────────────────────┼─────────┼─────────┼───────┼────────┼────────┼────────┤
│ alcohol + depression                   │ 250201  │  19168  │ 3607  │ 0.2597 │ 0.0027 │ 0.02

In [3]:
###################################################################################################
# Choose most appropriate concept names for best ranked pairs
# Print all possible MeSH entry names for concepts from top 10
# DONE
###################################################################################################
unique_conceptIDs = set()
for cmID, cmn in enumerate(cooc_measures_names):
    for chemDisePairIDs, _ in srt_cooc[cmn]:
        [chemID, diseID] = chemDisePairIDs.split('+')
        unique_conceptIDs.add( chemID )
        unique_conceptIDs.add( diseID )
for ID in unique_conceptIDs:
    print ID
    for n in MESHcodesToAllNames[ID]:
        print "\t", n
        
better_concept_names = [
    ['D003920', 'diabetes'], 
    ['D003866', 'depression'], 
    ['D009369', 'cancer'], 
    ['D000431', 'alcohol'], 
    ['D002251', 'carbon tetrachloride'], 
    ['D011020', 'pneumocystis pneumonia'],
    ['D010300', "Parkinson's disease"], 
    ['D010622', 'phencyclidine'], 
    ['D004409', 'abnormal movements'],         # orofacial dyskinesia / palpebral twitching
    ['D056486', 'toxic hepatitis'], 
    ['D006509', 'hepatitis B'],
    ['D001943', 'breast cancer'],
    ['D019259', 'lamivudine'],
    ['D000082', 'paracetamol'], 
    ['D007980', 'levodopa'], 
    ['D008774', 'methylphenidate'],
    ['D004317', 'doxorubicin'], 
    ['D003622', 'dapsone'], 
    ['D000244', 'adenosine diphosphate'],  # added manually, instead of ADP, based on search
    ['D007538', 'isoniazid']
]

D000244
	ADP
D007918
	leprosy
D003622
	dapsone
	Dapsone
D003920
	diabetes
	diabetes mellitus
	Diabetes mellitus
	diabetic
	Diabetic
D004317
	ADR
	adriamycin
	Adriamycin
	DOX
	doxorubicin
	Doxorubicin
D008774
	methylphenidate
	Methylphenidate
D001640
	BCC
	bicuculline
D002280
	basal cell carcinoma
D003866
	depressed
	depressed mood
	Depressed mood
	depression
	Depression
	depressions
	depressive
	depressive disorder
	depressive disorders
	depressive illness
	depressive symptoms
D001289
	ADHD
	attention-deficit/hyperactivity disorder
D012254
	ribavirin
D014376
	tuberculosis
D009369
	cancer
	Cancer
	cancers
	malignancies
	malignancy
	malignant tumor
	neoplasia
	neoplasms
	tumor
	tumors
	tumour
D006961
	hyperparathyroidism
D000431
	alcohol
	ALCOHOL
	ethanol
D002251
	carbon tetrachloride
	CCl4
D011020
	PCP
	Pneumocystis pneumonia
D005947
	glucose
D010300
	idiopathic Parkinson's disease
	parkinsonian
	Parkinson's disease
	Parkinson's Disease
	PD
D047508
	extensive hepatocellular necrosis
	ma

In [14]:
######################################################################################################
# Evaluate ranking similarity between the 4 measures
# (A) Top 10 ranking list overlap between various measures
######################################################################################################

# Construct top 10 ranking lists (of chemDisePairIDs) for each measure
ranking = []
for cmID, cmn in enumerate(cooc_measures_names):
    ranking.append( [] )
    for chemDisePairIDs, _ in srt_cooc[cmn]:
        ranking[cmID].append( chemDisePairIDs )
ranking = np.array(ranking)
print ranking.shape

tab_data = []
# Iterate over all pairs of measures
for cmID1 in range(len(cooc_measures_names)):
    tab_data.append( [] )
    for cmID2 in range(len(cooc_measures_names)):
        #print cmID1,cmID2
        cmn1 = cooc_measures_names[cmID1]
        cmn2 = cooc_measures_names[cmID2]
        N_shared = len( set(ranking[cmID1]).intersection(ranking[cmID2]) )
        #print N_shared
        tab_data[cmID1].append( N_shared )

tab_data = np.array(tab_data)
# fmt = ("", ".3f", ".3f", ".3f", ".3f")
print "\nNumber of chemical-disease pairs shared between two raknkings by various measures."
print tabulate(tab_data, 
               headers=cooc_measures_names, tablefmt='fancy_grid', showindex=cooc_measures_names,
               numalign='center')#, floatfmt=fmt)
print tabulate(tab_data, 
               headers=cooc_measures_names, tablefmt='latex_booktabs', showindex=cooc_measures_names,
               numalign='center')#, floatfmt=fmt)

(4, 10)

Number of chemical-disease pairs shared between two raknkings by various measures.
╒══════╤═══════╤════════╤═══════╤══════╕
│      │  SCC  │  NPMI  │  SCP  │  DC  │
╞══════╪═══════╪════════╪═══════╪══════╡
│ SCC  │  10   │   1    │   2   │  3   │
├──────┼───────┼────────┼───────┼──────┤
│ NPMI │   1   │   10   │   7   │  6   │
├──────┼───────┼────────┼───────┼──────┤
│ SCP  │   2   │   7    │  10   │  8   │
├──────┼───────┼────────┼───────┼──────┤
│ DC   │   3   │   6    │   8   │  10  │
╘══════╧═══════╧════════╧═══════╧══════╛
\begin{tabular}{lcccc}
\toprule
      &  SCC  &  NPMI  &  SCP  &  DC  \\
\midrule
 SCC  &  10   &   1    &   2   &  3   \\
 NPMI &   1   &   10   &   7   &  6   \\
 SCP  &   2   &   7    &  10   &  8   \\
 DC   &   3   &   6    &   8   &  10  \\
\bottomrule
\end{tabular}


In [47]:
######################################################################################################
# Evaluate similarity between the 4 rankings
# (B) Normalised Spearman's Footrule
######################################################################################################
# Kendall's tau / Spearman's Footrule
# http://theory.stanford.edu/~sergei/slides/www10-metrics.pdf
# Kendall: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kendalltau.html

from scipy.stats import kendalltau

# Spearman's Footrule: normalised to [0,1] ; 1 means identical rankings
def spearmanFootrule(a, b, norm=True):
    SF = 0.
    for i, x in enumerate(a):
        j = np.argwhere(b == x)[0][0]
        SF += abs(i-j)
    if norm:
        a_rev = np.flip(a, axis=0)
        SF = 1. - (SF / spearmanFootrule(a, a_rev, False))
    return SF

# FOR ALL
# Construct ranking lists (of chemDisePairIDs) for each measure
ranking = []
for cmID, cmn in enumerate(cooc_measures_names):
    ranking.append( [] )
    for chemDisePairIDs, _ in srt_cooc_all[cmn]:
        ranking[cmID].append( chemDisePairIDs )
# print ranking[0][4:8]
# print len(ranking[0]),len(ranking[1]),len(ranking[2]),len(ranking[3])
ranking = np.array(ranking)
print ranking.shape
# print ranking[0]
# print ranking[1]
# print kendalltau(ranking[0,:100], ranking[1,:100])

tab_data = []
tab_data2 = []
# Iterate over all pairs of measures
# for cmID1 in range(len(cooc_measures_names) - 1):
for cmID1 in range(len(cooc_measures_names)):
    tab_data.append( [] )
    tab_data2.append( [] )
#     for cmID2 in range(cmID1+1, len(cooc_measures_names)):
    for cmID2 in range(len(cooc_measures_names)):
        #print cmID1,cmID2
    
        cmn1 = cooc_measures_names[cmID1]
        cmn2 = cooc_measures_names[cmID2]
        
        if cmID1 == cmID2:
            KT = 1.
            p = 0.
            SF = 1.
        elif cmID1 < cmID2:
        
            # Kendall tau
            KT, p = kendalltau(ranking[cmID1], ranking[cmID2])

            # Spearman's Footrule
            SF = spearmanFootrule(ranking[cmID1], ranking[cmID2])
            
        else:
            KT = tab_data[cmID2][cmID1]
            p = None
            SF = tab_data2[cmID2][cmID1]
            
        #print ranking[cmID1]
        print cmn1, cmn2, "\t", KT, p, "\t\t\t", SF
        
        tab_data[cmID1].append( KT )
        tab_data2[cmID1].append( SF )

tab_data = np.array(tab_data)
fmt = ("", ".3f", ".3f", ".3f", ".3f")
print "\nKendall rank correlation coefficient:"
print tabulate(tab_data, 
               headers=cooc_measures_names, tablefmt='fancy_grid', showindex=cooc_measures_names,
               numalign='center', floatfmt=fmt)
print tabulate(tab_data, 
               headers=cooc_measures_names, tablefmt='latex_booktabs', showindex=cooc_measures_names,
               numalign='center', floatfmt=fmt)

tab_data2 = np.array(tab_data2)
fmt = ("", ".3f", ".3f", ".3f", ".3f")
print "\nSpearman's Footrule:"
print tabulate(tab_data2, 
               headers=cooc_measures_names, tablefmt='fancy_grid', showindex=cooc_measures_names,
               numalign='center', floatfmt=fmt)
print tabulate(tab_data2, 
               headers=cooc_measures_names, tablefmt='latex_booktabs', showindex=cooc_measures_names,
               numalign='center', floatfmt=fmt)

(4, 46137)
SCC SCC 	1.0 0.0 			1.0
SCC NPMI 	-0.00105516144744 0.733892218023 			0.42466676087
SCC SCP 	0.0051792694866 0.0951846927262 			0.608996460758
SCC DC 	0.00114794269714 0.711498034512 			0.610066848632
NPMI SCC 	-0.00105516144744 None 			0.42466676087
NPMI NPMI 	1.0 0.0 			1.0
NPMI SCP 	0.00129812192733 0.675779128277 			0.783353286015
NPMI DC 	0.00166228452704 0.592265500138 			0.651308544117
SCP SCC 	0.0051792694866 None 			0.608996460758
SCP NPMI 	0.00129812192733 None 			0.783353286015
SCP SCP 	1.0 0.0 			1.0
SCP DC 	0.000288382382279 0.925974087867 			0.744877459659
DC SCC 	0.00114794269714 None 			0.610066848632
DC NPMI 	0.00166228452704 None 			0.651308544117
DC SCP 	0.000288382382279 None 			0.744877459659
DC DC 	1.0 0.0 			1.0

Kendall rank correlation coefficient:
╒══════╤════════╤════════╤═══════╤═══════╕
│      │  SCC   │  NPMI  │  SCP  │  DC   │
╞══════╪════════╪════════╪═══════╪═══════╡
│ SCC  │ 1.000  │ -0.001 │ 0.005 │ 0.001 │
├──────┼────────┼────────┼───────

In [6]:
######################################################################################################
# Find the top 10 ranked chem-dise pairs in the dataset of chemically-induced diseases (CID) (BioCreative V CDR)
######################################################################################################

######################################################
# Collect all CIDs (chemID+diseID) from BioCreative V
CIDs = [] # list of "chemID+diseID" found
# Iterate over all sets: train, development, test
for file_name in glob.glob('./bmip-2018-master/BC5-CDR/original-data/CDR_*.txt'):
    with open(file_name) as f:  
        # Iterate over all lines of the file
        while True: 
            # Read line , check if CID
            rl = f.readline()
            rlwn = rl.split('\n')[0] # read line skipping the new line character
            rlwn_split = rlwn.split('\t') # split the line
            if len(rlwn_split) > 1:
                if rlwn_split[1] == 'CID':
                    #print rlwn_split
                    CIDs.append( rlwn_split[2] + '+' + rlwn_split[3] )
            # End of file
            if rl == '':
                break
print CIDs[1:4]
print "Total # CIDs:", len(CIDs) # 3116
print 

######################################################
# Find intersection with top-ranked chem-dise pairs

all_chemDise_pairIDs = [] # make list of all pair IDs: 40 in total
for cmID, cmn in enumerate(cooc_measures_names):
    for chemDisePairIDs, _ in srt_cooc[cmn]:
        all_chemDise_pairIDs.append( chemDisePairIDs )
unique_chemDise_pairIDs = np.unique(all_chemDise_pairIDs)

print "Confirmed chemically-induced diseases:"
confirmed_pairsNames = []
for p in unique_chemDise_pairIDs:
    if p in CIDs:
        [chemID, diseID] = p.split('+')
        name = MESHcodesToNames[chemID] + ' + ' + MESHcodesToNames[diseID] 
        print "\t", name, "\t\t\t", "{:}\t{:}".format(chemID,diseID)
        confirmed_pairsNames.append(name)
    
tab_data = np.array([confirmed_pairsNames]).T
tab_header = ['Chemical ($c$) + Disease ($d$)']
print tabulate(tab_data, 
               headers=tab_header, tablefmt='fancy_grid', 
               numalign='center')
print tabulate(tab_data, 
               headers=tab_header, tablefmt='latex_booktabs', 
               numalign='center')

# Calculate overlap between top 10 and set of confirmed => compare 4 measures
print "\n# chem-dise pairs shared between the top 10 ranked and confirmed CIDs"
shared_cnts = []
for cmID, cmn in enumerate(cooc_measures_names):
    cnt = 0
    for chemDisePairIDs, _ in srt_cooc[cmn]:
        if chemDisePairIDs in CIDs:
            cnt += 1
            [chemID, diseID] = chemDisePairIDs.split('+')
            name = MESHcodesToNames[chemID] + ' + ' + MESHcodesToNames[diseID] 
    print cmn, ": ", cnt
    shared_cnts.append( cnt )
    
tab_data = np.array([shared_cnts]).T
tab_header = ['Co-occurrence measure', '# shared chemically-induced diseases']
print tabulate(tab_data, 
               headers=tab_header, tablefmt='fancy_grid', showindex=cooc_measures_names,
               numalign='center')
print tabulate(tab_data, 
               headers=tab_header, tablefmt='latex_booktabs', showindex=cooc_measures_names,
               numalign='center')

['D007213+D007022', 'D016572+D057049', 'D000305+D012595']
Total # CIDs: 3116

Confirmed chemically-induced diseases:
	paracetamol + toxic hepatitis 			D000082	D056486
	alcohol + depression 			D000431	D003866
	alcohol + toxic hepatitis 			D000431	D056486
	carbon tetrachloride + toxic hepatitis 			D002251	D056486
	estrogen + breast cancer 			D004967	D001943
	levodopa + abnormal movements 			D007980	D004409
	streptozotocin + diabetes 			D013311	D003920
╒════════════════════════════════════════╕
│ Chemical ($c$) + Disease ($d$)         │
╞════════════════════════════════════════╡
│ paracetamol + toxic hepatitis          │
├────────────────────────────────────────┤
│ alcohol + depression                   │
├────────────────────────────────────────┤
│ alcohol + toxic hepatitis              │
├────────────────────────────────────────┤
│ carbon tetrachloride + toxic hepatitis │
├────────────────────────────────────────┤
│ estrogen + breast cancer               │
├─────────────────────────────

In [8]:
###################################################################################################
# Extract unique chemical-disease pairs (CDPs) from all top 10 rankings
# Examine chemical-disease relation type: print table & calculate what kind of relations are best found by which measure
###################################################################################################
from collections import Counter

all_chemDise_pairs = []
for cmID, cmn in enumerate(cooc_measures_names):
    for chemDisePairIDs, _ in srt_cooc[cmn]:
        [chemID, diseID] = chemDisePairIDs.split('+')
        name = MESHcodesToNames[chemID] + ' + ' + MESHcodesToNames[diseID] 
        all_chemDise_pairs.append( name )
        
# print sorted( dict(Counter(all_chemDise_pairs)).items(), key=lambda x: x[1], reverse=True)
srt_unique_chemDise_pairs = sorted( dict(Counter(all_chemDise_pairs)).items(), key=lambda x: x[1], reverse=True)
srt_unique_chemDise_pairs_names = []
for x,y in srt_unique_chemDise_pairs:
    print '{:<60}  {:>20}'.format(x, y)
    srt_unique_chemDise_pairs_names.append( x )
print 
print "# unique pairs = ", len(srt_unique_chemDise_pairs)
print "ONLY ........... levodopa + abnormal movements ............... appears in each of the four top 10 rankings "
print "6 pairs shared among 3 rankings"
print "6 pairs shared among 3 rankings"
print "3 pairs shared among 2 rankings"

#################################################
# Print table with CDP relation types

relation_types = ['CAUSES', 'TREATS', 
                  'DETECTS', 'INCREASES RISK OF', 'ACCOMPANIES', 'UNKNOWN'
                 ]

relations_for_CDPs = [
    ['CAUSES,TREATS', 'MD'], 
    ['CAUSES', 'MD'], 
    ['TREATS', 'MD'], 
    ['TREATS', 'MD'], 
    ['TREATS', 'MD'], 
    ['TREATS', 'MD'], 
    ['UNKNOWN', 'MD'],              ###
    ['TREATS', 'MD'], 
    ['DETECTS', 'MD'], 
    ['CAUSES', 'MD'], 
    ['INCREASES RISK OF', 'MD'],                 #
    ['TREATS', 'MD'], 
    ['TREATS', 'MD'], 
    ['CAUSES', 'MD'], 
    ['TREATS', 'MD'], 
    ['UNKNOWN', 'MD'],                  ###
    ['CAUSES', 'MD'], 
    ['TREATS', 'MD'], 
    ['CAUSES', 'MD'], 
    ['CAUSES', 'MD'], 
    ['ACCOMPANIES', 'MD'],                         #
    ['CAUSES,ACCOMPANIES', 'MD'],               #
]
relations_for_CDPs = np.array(relations_for_CDPs)
tab_data = [[x[0]] for x in relations_for_CDPs]
# tab_header = ['', 'Relation type', 'Reference']
tab_header = ['', 'Relation type']
# fmt = ("", ".3f", ".3f", ".3f", ".3f")
print "\nChemical-disease relations of the unique chemical-disease pairs (CDPs) (aggregated from all 4 top 10 rankings)."
print tabulate(tab_data, 
               headers=tab_header, tablefmt='fancy_grid', showindex=srt_unique_chemDise_pairs_names,
               numalign='center')#, floatfmt=fmt)
print tabulate(tab_data, 
               headers=tab_header, tablefmt='latex_booktabs', showindex=srt_unique_chemDise_pairs_names,
               numalign='center')#, floatfmt=fmt)

######################################################################
# Print table with CDP relation types COUNTS by various cooc measures

tab_data = np.zeros( (len(cooc_measures_names), len(relation_types)) )

for cmID, cmn in enumerate(cooc_measures_names): # over measures
    for rtID, rt in enumerate(relation_types): # over relation types
        cnt = 0
        for relID, relations in enumerate(relations_for_CDPs[:,0]): # over all relations for ordered unique CDPs
            for relation in relations.split(','):
                if rt == relation: # found CDP of that relation type
                    for chemDisePairIDs, _ in srt_cooc[cmn]:
                        [chemID, diseID] = chemDisePairIDs.split('+')
                        name = MESHcodesToNames[chemID] + ' + ' + MESHcodesToNames[diseID] 
                        if name == srt_unique_chemDise_pairs_names[relID]: # if found in top 10 for the given measure
                            cnt += 1
        tab_data[cmID, rtID] = cnt # set count
    
tab_header = ['Co-occurrence measure'] + relation_types
# fmt = ("", ".3f", ".3f", ".3f", ".3f")
print "\nChemical-disease relations of the unique chemical-disease pairs (CDPs) (aggregated from all 4 top 10 rankings)."
print tabulate(tab_data, 
               headers=tab_header, tablefmt='fancy_grid', showindex=cooc_measures_names,
               numalign='center')#, floatfmt=fmt)
print tabulate(tab_data, 
               headers=tab_header, tablefmt='latex_booktabs', showindex=cooc_measures_names,
               numalign='center')#, floatfmt=fmt)

levodopa + abnormal movements                                                    4
carbon monoxide + poisoning                                                      3
ribavirin + hepatitis C                                                          3
lamivudine + hepatitis B                                                         3
levodopa + Parkinson's disease                                                   3
methylphenidate + attention-deficit/hyperactivity disorder                       3
phencyclidine + pneumocystis pneumonia                                           3
dapsone + leprosy                                                                2
adenosine diphosphate + platelet aggregations                                    2
carbon tetrachloride + toxic hepatitis                                           2
estrogen + breast cancer                                                         1
cisplatinum + cancer                                                             1
ison

In [18]:
# Kendall tau testing
# NOT USED

print kendalltau( ['a', 'b', 'c'], ['a', 'b', 'c'] )
print kendalltau( ['a', 'b', 'c'], ['a', 'c', 'b'] )

print kendalltau( ['a', 'b', 'c'], ['b', 'c', 'a'] )
print kendalltau( ['a', 'b', 'c'], ['b', 'a', 'c'] )

print kendalltau( ['a', 'b', 'c'], ['c', 'a', 'b'] )
print kendalltau( ['a', 'b', 'c'], ['c', 'b', 'a'] )


KendalltauResult(correlation=1.0, pvalue=0.1171850871981381)
KendalltauResult(correlation=0.33333333333333337, pvalue=0.60150813444058993)
KendalltauResult(correlation=-0.33333333333333337, pvalue=0.60150813444058993)
KendalltauResult(correlation=0.33333333333333337, pvalue=0.60150813444058993)
KendalltauResult(correlation=-0.33333333333333337, pvalue=0.60150813444058993)
KendalltauResult(correlation=-1.0, pvalue=0.1171850871981381)
